In [1]:
!pip install -q psycopg2 # library for postrgres connector
from sqlalchemy import create_engine
import pandas as pd
import duckdb
duckcon = duckdb.connect()

In [2]:
# connect to engine. This assumes a DB name of saddl, and root user without a pw
engine = create_engine('postgresql://:@localhost:5432/saddl')

In [3]:
with engine.connect() as con:
    with open('../db-export/0-0-1-prerelease/schema.sql', mode='r') as f:
        cmds = []
        for line in f.readlines():
            if 'CREATE TABLE' in line:
                line = line.strip().replace('BIGINT', 'INT').replace('FLOAT', 'REAL').replace('DOUBLE', 'REAL')
                print(line)
                con.execute(line.split('(')[0].replace('CREATE TABLE', 'DROP TABLE IF EXISTS'))
                result = con.execute(line)

CREATE TABLE meta(htid VARCHAR, "access" VARCHAR, rights VARCHAR, ht_bib_key VARCHAR, description VARCHAR, source VARCHAR, source_bib_num VARCHAR, oclc_num VARCHAR, isbn VARCHAR, issn VARCHAR, lccn VARCHAR, title VARCHAR, imprint VARCHAR, rights_reason_code VARCHAR, rights_timestamp VARCHAR, us_gov_doc_flag INT, rights_date_used REAL, pub_place VARCHAR, lang VARCHAR, bib_fmt VARCHAR, collection_code VARCHAR, content_provider_code VARCHAR, responsible_entity_code VARCHAR, digitization_agent_code VARCHAR, access_profile_code VARCHAR, author VARCHAR, page_count VARCHAR);
CREATE TABLE clean_predictions(target VARCHAR, candidate VARCHAR, swsm REAL, swde REAL, wp_dv REAL, partof REAL, contains REAL, "OVERLAPS" REAL, author REAL, simdiff REAL, grsim REAL, randdiff REAL, relatedness REAL, count INTEGER);
CREATE TABLE work_predictions(target INT, candidate INT, swsm REAL, swde REAL, wp_dv REAL, partof REAL, contains REAL, "overlaps" REAL, author REAL, simdiff REAL, grsim REAL, randdiff REAL, co

In [4]:
duckcon.execute(f"SELECT * FROM '{csv_tmp_path}' LIMIT 1").fetch_df()

NameError: name 'csv_tmp_path' is not defined

Paths in load.sql should correctly point to the DB export and be absolute.

In [8]:
import os
import time
with engine.connect() as con:
    with open('../db-export/0-0-1-prerelease/load.sql', mode='r') as f:
        copy_cmds = []
        print("THIS COPY CMD doesn't run from Python's SQL connector, copy and run it in the command line.")
        print("="*40)
        for copy_cmd in f.readlines():
            if 'COPY' not in copy_cmd:
                continue
            a, b = copy_cmd.split(' FROM \'')
            _, table_name = a.split()
            parquet_location = b.split("' (FORMAT")[0].strip("'")
            csv_tmp_path = parquet_location.replace('parquet', 'csv')
            #pd.read_parquet(parquet_location).to_csv(csv_tmp_path, index=False, header=True)
            # use duckdb instead
            duckcon.execute(f"COPY '{parquet_location}' TO '{csv_tmp_path}' WITH ( DELIMITER ',', HEADER 1);")
            pq_cmd = f"COPY {table_name} FROM '{csv_tmp_path}' DELIMITER ',' CSV HEADER;"
            print(pq_cmd)
            #con.execute(pq_cmd)
            #os.remove(csv_tmp_path)

THIS COPY CMD doesn't run from Python's SQL connector, copy and run it in the command line.
COPY clusters FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/7_clusters.csv' DELIMITER ',' CSV HEADER;
COPY clean_predictions FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/1_clean_predictions.csv' DELIMITER ',' CSV HEADER;
COPY work_predictions FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/2_work_predictions.csv' DELIMITER ',' CSV HEADER;
COPY meta FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/0_meta.csv' DELIMITER ',' CSV HEADER;
COPY manifestation_predictions FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/6_manifestation_predictions.csv' DELIMITER ',' CSV HEADER;
COPY manifestation_stats FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/3_manifestation_stats.csv' DELIMI

if you have users with privileges granted to the tables that you just dropped and recreated, you need to run this
```sql
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO saddl;
GRANT ALL PRIVILEGES ON ALL SEQUENCES IN SCHEMA public TO saddl;
```

## ADD PRIMARY KEY

`ALTER TABLE` locks the table, so make sure there's nothing else blocking your query.

https://stackoverflow.com/questions/26489244/how-to-detect-query-which-holds-the-lock-in-postgres

https://stackoverflow.com/questions/35319597/how-to-stop-kill-a-query-in-postgresql

In [9]:
with engine.connect() as con:
    con.execute('ALTER TABLE work_stats ADD PRIMARY KEY (work_id);')
    con.execute('ALTER TABLE manifestation_stats ADD PRIMARY KEY (man_id);')
    con.execute('ALTER TABLE meta ADD PRIMARY KEY (htid);')
    con.execute('ALTER TABLE clusters ADD PRIMARY KEY (htid);')

## CREATE INDEX

This makes a huge difference.

In [10]:
with engine.connect() as con:
    con.execute('CREATE INDEX target_idx ON clean_predictions (target);')
    con.execute('CREATE INDEX candidate_idx ON clean_predictions (candidate);')

    con.execute('CREATE INDEX target_work_idx ON work_predictions (target);')
    con.execute('CREATE INDEX candidate_work_idx ON work_predictions (candidate);')

    con.execute('CREATE INDEX target_man_idx ON manifestation_predictions (target);')
    con.execute('CREATE INDEX candidate_man_idx ON manifestation_predictions (candidate);')

    con.execute('CREATE INDEX work_cluster_idx ON clusters (work_id);')
    con.execute('CREATE INDEX man_cluster_idx ON clusters (man_id);')

## Create Search vector column and index

Currently, this is just title and author concatenated. Can be improved with weighting or field-specific search, or text highlighting.
https://www.postgresql.org/docs/9.5/textsearch-tables.html.

Some notes:
- `coalesce(title, '')` ensures that fields are still concatenated when NULL.

In [11]:
with engine.connect() as con:
    con.execute('ALTER TABLE meta ADD COLUMN textsearch tsvector;')
    con.execute('''UPDATE meta SET textsearch =
     to_tsvector('english', coalesce(title,'') || ' ' || coalesce(author,''));
    ''')
    con.execute('CREATE INDEX meta_search_idx ON meta USING GIN (textsearch);')

## Test

In [12]:
with engine.connect() as con:
    result = con.execute('SELECT * FROM work_stats LIMIT 1')
result.all()

[(730141, 1, 0, 0, 0.0, 0.0, True, None, None, None, None)]

In [13]:
pd.read_sql('SELECT * FROM work_stats LIMIT 1', engine)

,work_id,label_count,gov_count,serial_count,gov_prop,serial_prop,include,best_centroid,best_centroid_pd,best_median,best_median_pd
0,730141,1,0,0,0.0,0.0,True,None,None,None,None
